In [1]:
import itertools
import pandas as pd
import numpy as np
import os
import csv
import threading
import multiprocessing as mp
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
from Core_LP import single_opt2
import paper_classes as pc
import post_proc as pp
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'Core_LP'

In [29]:
combinations=dict({'Capacity':7,'App_comb':0,'Tech':'NMC','PV_nom':4.8,'country':'CH','cases':'mean','conf':6,'house_type':'SFH15','HP':'AS','hh':110143956137})
#id_dwell=str(combinations['hh'])
#App_comb=[False,True,False,False]

In [30]:
import pickle
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [31]:
id_dwell=str(int(combinations['hh']))
print(id_dwell)
[clusters,PV,App_comb_df,conf,house_types,hp_types,rad_types]=pp.get_table_inputs()

PV_nom=4.8
quartile=50
App_comb=[False,True,False,False]
#####################################################
design_param=load_obj('../Input/dict_design_oct')
aging=True
Inverter_power=round(PV_nom/1.2,1)
Curtailment=0.5
Inverter_Efficiency=0.95
Converter_Efficiency_HP=0.98
dt=0.25
Capacity_tariff=9.39*12/365
nyears=1
days=365
testing=False
cooling=False
week=1
######################################################

filename_el=Path('../Input/Electricity_demand_supply_2017.csv')
filename_heat=Path('../Input/preprocessed_heat_demand_2_new_Oct.csv')
filename_prices=Path('../Input/Prices_2017.csv')

fields_el=['index',id_dwell,'E_PV']
new_cols=['E_demand', 'E_PV']

df_el = pd.read_csv(filename_el,engine='python',sep=',|;',index_col=0,
                    parse_dates=[0],infer_datetime_format=True, usecols=fields_el)

if np.issubdtype(df_el.index.dtype, np.datetime64):
    df_el.index=df_el.index.tz_localize('UTC').tz_convert('Europe/Brussels')
else:
    df_el.index=pd.to_datetime(df_el.index,utc=True)
    df_el.index=df_el.index.tz_convert('Europe/Brussels')

df_el.columns=new_cols

if (combinations['house_type']=='SFH15')| (combinations['house_type']=='SFH45'):
    aux_name='SFH15_45'
else:
    aux_name='SFH100'

fields_heat=['index','Set_T','Temp', combinations['house_type']+'_kWh','DHW_kWh', 'Temp_supply_'+aux_name,'Temp_supply_'+aux_name+'_tank',
            'COP_'+combinations['house_type'],'hp_'+combinations['house_type']+'_el_cons','COP_'+combinations['house_type']+'_DHW',
             'hp_'+combinations['house_type']+'_el_cons_DHW','COP_'+combinations['house_type']+'_tank',
             'hp_'+combinations['house_type']+'_tank_el_cons']
new_cols=['Set_T','Temp', 'Req_kWh','Req_kWh_DHW','Temp_supply','Temp_supply_tank','COP_SH','COP_tank','COP_DHW',
          'hp_sh_cons','hp_tank_cons','hp_dhw_cons']
df_heat=pd.read_csv(filename_heat,engine='python',sep=';',index_col=[0],
                    parse_dates=[0],infer_datetime_format=True, usecols=fields_heat)
df_heat.columns=new_cols


if np.issubdtype(df_heat.index.dtype, np.datetime64):
    df_heat.index=df_heat.index.tz_localize('UTC').tz_convert('Europe/Brussels')
else:
    df_heat.index=pd.to_datetime(df_heat.index,utc=True)
    df_heat.index=df_heat.index.tz_convert('Europe/Brussels')

fields_prices=['index', 'Price_flat', 'Price_DT', 'Export_price', 'Price_flat_mod',
'Price_DT_mod']
df_prices=pd.read_csv(filename_prices,engine='python',sep=',|;',index_col=[0],
                    parse_dates=[0],infer_datetime_format=True ,usecols=fields_prices)

if np.issubdtype(df_prices.index.dtype, np.datetime64):
    df_prices.index=df_prices.index.tz_localize('UTC').tz_convert('Europe/Brussels')
else:
    df_prices.index=pd.to_datetime(df_prices.index,utc=True)
    df_prices.index=df_prices.index.tz_convert('Europe/Brussels')

data_input=pd.concat([df_el,df_heat,df_prices],axis=1,copy=True,sort=False)
#skip the first DHW data since cannot be produced simultaneously with SH
data_input.loc[(data_input.index.hour<2),'Req_kWh_DHW']=0
T_var=data_input.Temp.resample('1d').mean()
data_input.loc[:,'E_PV']=data_input.loc[:,'E_PV']*PV_nom
data_input['T_var']=T_var
data_input.T_var=data_input.T_var.fillna(method='ffill')
data_input['Cooling']=0
data_input.loc[((data_input.index.month<=4)|(data_input.index.month>=10))&(data_input.Req_kWh<0),'Req_kWh']=0
if cooling:
    data_input.loc[(data_input.index.month>4)&(data_input.index.month<10)&(data_input.T_var>20),'Cooling']=1#is T_var>20 then we need to cool only
    data_input.loc[(data_input.Cooling==1)&(data_input.Req_kWh>0),'Req_kWh']=0#if we should cool then ignore the heating requirements
    data_input.loc[(data_input.Cooling==1),'Req_kWh']=abs(data_input.loc[(data_input.Cooling==1),'Req_kWh'])

data_input.loc[(data_input.index.month>4)&(data_input.index.month<10)&(data_input.Cooling==0),'Req_kWh']=0#if we should heat then ignore the cooling requirements
data_input['Temp']=data_input['Temp']+273.15
data_input['Set_T']=data_input['Set_T']+273.15
data_input['Temp_supply']=data_input['Temp_supply']+273.15
data_input['Temp_supply_tank']=data_input['Temp_supply_tank']+273.15
data_input.loc[data_input.index.dayofyear==120,'Req_kWh']=0
data_input.loc[data_input.index.dayofyear==274,'Req_kWh']=0
data_input.loc[(data_input.index.dayofyear<120)|(data_input.index.dayofyear>274),'season']=0#'heating'
data_input.loc[data_input.index.dayofyear==120,'season']=1#'transition_heating_cooling'
data_input.loc[(data_input.index.dayofyear>120)&(data_input.index.dayofyear<274),'season']=2#'cooling'
data_input.loc[data_input.index.dayofyear==274,'season']=3#'transition_cooling_heating'
if data_input[((data_input.index.dayofyear<120)|(data_input.index.dayofyear>274))&(data_input.Temp_supply==data_input.Temp_supply_tank)].empty==False:
    data_input.loc[((data_input.index.dayofyear<120)|(data_input.index.dayofyear>274))&(data_input.Temp_supply==data_input.Temp_supply_tank),'Temp_supply_tank']+=1.5

if testing:
    data_input=data_input[data_input.index.week==week]
    nyears=1
    days=7
    ndays=7

print('##############')
print('load parameters')
conf=combinations['conf']
print('conf')
print(conf)

#configuration=[Batt,HP,TS,DHW]
#if all false, only PV
conf_aux=[False,True,False,False]#[Batt,HP,TS,DHW]

if conf<4:#No battery
    #print('inside batt')
    Converter_Efficiency_Batt=1
else:
    conf_aux[0]=True
    Converter_Efficiency_Batt=0.98

if (conf!=0)&(conf!=1)&(conf!=4)&(conf!=5):#TS present
    #print('inside TS')
    conf_aux[2]=True
    if (combinations['house_type']=='SFH15')|(combinations['house_type']=='SFH45'):
        tank_sh=pc.heat_storage_tank(mass=9578.54,surface=18.85)# For a 9600 liter tank with 3 m height and 2 m diameter 
        T_min_cooling=285.15#12°C
    else:
        tank_sh=pc.heat_storage_tank(mass=1500,surface=6) # For a 1500 liter tank with 1.426 m height and 1.25 diameter 
        T_min_cooling=285.15#12°C
else:#No TS
    tank_sh=pc.heat_storage_tank(mass=0, surface=0.41)# For a 50 liter tank with .26 m height and .25 diameter
    T_min_cooling=0

if (conf==1)|(conf==3)|(conf==5)|(conf==7):#DHW present
    #print('inside DHW')
    conf_aux[3]=True
    tank_dhw=pc.heat_storage_tank(mass=200, t_max=60+273.15, t_min=40+273.15,surface=1.6564) # For a 200 liter tank with 0.95 m height and .555 diameter
    if (conf==1)|(conf==5):
        if combinations['house_type']=='SFH15':
            tank_sh=pc.heat_storage_tank(mass=0, surface=1.209)# For a 100 liter tank with  .52m height and .25 diameter
        elif combinations['house_type']=='SFH45':
            tank_sh=pc.heat_storage_tank(mass=0, surface=1.913)# For a 200 liter tank with .52 m height and .35 diameter
        elif combinations['house_type']=='SFH100':
            tank_sh=pc.heat_storage_tank(mass=0, surface=3.2)# For a 50 liter tank with .52 m height and .5 diameter
else:#No DHW
    tank_dhw=pc.heat_storage_tank(mass=1, t_max=0, t_min=0,specific_heat_dhw=0,U_value_dhw=0,surface_dhw=0)#null

ndays=days*nyears
print(data_input.head())
if combinations['HP']=='AS':
    if combinations['house_type']=='SFH15':
        Backup_heater=design_param['bu_15']+2
        hp=pc.HP_tech(technology='ASHP',power=design_param['hp_15'])
    elif combinations['house_type']=='SFH45':
        Backup_heater=design_param['bu_45']+4
        hp=pc.HP_tech(technology='ASHP',power=design_param['hp_45'])
    else:
        Backup_heater=design_param['bu_100']+17
        hp=pc.HP_tech(technology='ASHP',power=design_param['hp_100'])
elif combinations['HP']=='GSHP':
    #TODO
    pass
param={'conf':conf_aux,
'aging':aging,'Inv_power':Inverter_power,
'Curtailment':Curtailment,'Inverter_eff':Inverter_Efficiency,
'Converter_Efficiency_HP':Converter_Efficiency_HP,
'Converter_Efficiency_Batt':Converter_Efficiency_Batt,
'delta_t':dt,'nyears':nyears,'T_min_cooling':T_min_cooling,
'days':days,'ndays':ndays,'hp':hp,'tank_dhw':tank_dhw,'tank_sh':tank_sh,
'Backup_heater':Backup_heater,'Capacity':combinations['Capacity'],'Tech':combinations['Tech'],
'App_comb':App_comb,'cases':combinations['cases'],'ht':combinations['house_type'],
'HP_type':combinations['HP'],'testing':testing, 'Cooling_ind':cooling,'name':str(id_dwell)+'_'+combinations['country']+'_PV'+str(quartile),
'PV_nom':PV_nom,'Capacity_tariff':Capacity_tariff}

110143956137
##############
load parameters
conf
6
                           E_demand  E_PV   Set_T    Temp  Req_kWh  \
index                                                                
2017-01-01 00:00:00+01:00      0.06   0.0  293.15  271.15      0.0   
2017-01-01 00:15:00+01:00      0.06   0.0  293.15  271.15      0.0   
2017-01-01 00:30:00+01:00      0.07   0.0  293.15  271.15      0.0   
2017-01-01 00:45:00+01:00      0.06   0.0  293.15  271.15      0.0   
2017-01-01 01:00:00+01:00      0.06   0.0  293.15  271.05      0.0   

                           Req_kWh_DHW  Temp_supply  Temp_supply_tank  COP_SH  \
index                                                                           
2017-01-01 00:00:00+01:00          0.0   303.826003        307.404292    3.13   
2017-01-01 00:15:00+01:00          0.0   303.826003        307.404292    3.13   
2017-01-01 00:30:00+01:00          0.0   303.826003        307.404292    3.13   
2017-01-01 00:45:00+01:00          0.0   303.826003  

In [35]:
data_input[(data_input.index.dayofyear>120)&(data_input.index.dayofyear<=274)&(data_input.Req_kWh>0)].index.dayofyear.unique()

Int64Index([], dtype='int64', name='index')

In [34]:
if param['nyears']>1:
    data_input=pd.DataFrame(pd.np.tile(pd.np.array(data_input).T,
                           param['nyears']).T,columns=data_input.columns)
print('#############pool################')
print('the DHW tank mass is {}: '.format(param['tank_dhw'].mass))
print('the SH tank mass is {}: '.format(param['tank_sh'].mass))
print('the SH tank mass is {}: '.format(param['tank_sh'].specific_heat))
print('Configuration is {}'.format(param['conf']))
print('Application is {}'.format(param['App_comb']))
print(data_input.sum().round(1))

#############pool################
the DHW tank mass is 1: 
the SH tank mass is 9578.54: 
the SH tank mass is 0.00116: 
Configuration is [True, True, True, False]
Application is [False, True, False, False]
E_demand                4224.1
E_PV                    6721.9
Set_T               10271976.0
Temp                 9967762.8
Req_kWh                 3814.7
Req_kWh_DHW             3602.5
Temp_supply         10437528.9
Temp_supply_tank    10493418.1
COP_SH                228826.3
COP_tank              224040.4
COP_DHW               101801.6
hp_sh_cons             31054.0
hp_tank_cons           31885.5
hp_dhw_cons            57789.2
Price_flat             10161.6
Price_DT                7576.2
Export_price            1611.9
Price_flat_mod          6012.9
Price_DT_mod            5985.2
T_var                 396586.8
Cooling                    0.0
season                 29760.0
T_aux_supply        10787928.9
dtype: float64


In [36]:
data_input.keys()

Index(['E_demand', 'E_PV', 'Set_T', 'Temp', 'Req_kWh', 'Req_kWh_DHW',
       'Temp_supply', 'Temp_supply_tank', 'COP_SH', 'COP_tank', 'COP_DHW',
       'hp_sh_cons', 'hp_tank_cons', 'hp_dhw_cons', 'Price_flat', 'Price_DT',
       'Export_price', 'Price_flat_mod', 'Price_DT_mod', 'T_var', 'Cooling',
       'season', 'T_aux_supply'],
      dtype='object')

In [32]:
data_input[data_input.index.dayofyear>118].round(2).head()

,E_demand,E_PV,Set_T,Temp,Req_kWh,Req_kWh_DHW,Temp_supply,Temp_supply_tank,COP_SH,COP_tank,...,hp_tank_cons,hp_dhw_cons,Price_flat,Price_DT,Export_price,Price_flat_mod,Price_DT_mod,T_var,Cooling,season
index,,,,,,,,,,,,,,,,,,,,,
2017-04-29 00:00:00+02:00,0.64,0.0,293.15,274.75,0.02,0.0,299.64,301.84,4.12,4.12,...,1.2,2.0,0.29,0.16,0.04,0.17,0.13,6.76,0,0.0
2017-04-29 00:15:00+02:00,0.33,0.0,293.15,274.75,0.02,0.0,299.65,301.85,4.12,4.12,...,1.2,2.0,0.29,0.16,0.04,0.17,0.13,6.76,0,0.0
2017-04-29 00:30:00+02:00,0.18,0.0,293.15,274.75,0.02,0.0,299.67,301.87,4.12,4.12,...,1.2,2.0,0.29,0.16,0.04,0.17,0.13,6.76,0,0.0
2017-04-29 00:45:00+02:00,0.42,0.0,293.15,274.75,0.02,0.0,299.68,301.88,4.12,4.12,...,1.2,2.0,0.29,0.16,0.04,0.17,0.13,6.76,0,0.0
2017-04-29 01:00:00+02:00,0.52,0.0,293.15,274.05,0.02,0.0,299.69,301.90,4.12,4.12,...,1.2,2.0,0.29,0.16,0.04,0.17,0.13,6.76,0,0.0


In [33]:
[df,aux_dict]=single_opt2(param,data_input)

@@@@@@@@@@@@@@@@@@@@@@@@@@
single opt
bf
af
%%%%%%%%% Optimizing %%%%%%%%%%%%%%%
0{0: 0.29000000000000004, 1: 0.29000000000000004, 2: 0.29000000000000004, 3: 0.29000000000000004, 4: 0.29000000000000004, 5: 0.29000000000000004, 6: 0.29000000000000004, 7: 0.29000000000000004, 8: 0.29000000000000004, 9: 0.29000000000000004, 10: 0.29000000000000004, 11: 0.29000000000000004, 12: 0.29000000000000004, 13: 0.29000000000000004, 14: 0.29000000000000004, 15: 0.29000000000000004, 16: 0.29000000000000004, 17: 0.29000000000000004, 18: 0.29000000000000004, 19: 0.29000000000000004, 20: 0.29000000000000004, 21: 0.29000000000000004, 22: 0.29000000000000004, 23: 0.29000000000000004, 24: 0.29000000000000004, 25: 0.29000000000000004, 26: 0.29000000000000004, 27: 0.29000000000000004, 28: 0.29000000000000004, 29: 0.29000000000000004, 30: 0.29000000000000004, 31: 0.29000000000000004, 32: 0.29000000000000004, 33: 0.29000000000000004, 34: 0.29000000000000004, 35: 0.29000000000000004, 36: 0.29000000000000004, 37

ApplicationError: Solver (cplex) did not exit normally

In [60]:
df1=pd.read_csv('../Output/aggregated_results_jan.csv',engine='python',sep=',|;')

In [61]:
df1.keys()

Index(['E_PV_batt', 'E_PV_bu', 'E_PV_budhw', 'E_PV_curt', 'E_PV_grid',
       'E_PV_hp', 'E_PV_hpdhw', 'E_PV_load', 'E_batt_bu', 'E_batt_budhw',
       'E_batt_hp', 'E_batt_hpdhw', 'E_batt_load', 'E_bu', 'E_budhw', 'E_char',
       'E_cons', 'E_dis', 'E_grid_batt', 'E_grid_bu', 'E_grid_budhw',
       'E_grid_hp', 'E_grid_hpdhw', 'E_grid_load', 'E_hp', 'E_hpdhw',
       'E_loss_Batt', 'E_loss_conv', 'E_loss_inv', 'E_loss_inv_PV',
       'E_loss_inv_batt', 'E_loss_inv_grid', 'Q_dhwst_hd', 'Q_hp_sh',
       'Q_hp_ts', 'Q_loss_dhwst', 'Q_loss_ts', 'Q_ts', 'Q_ts_delta', 'Q_ts_sh',
       'T_dhwst', 'T_ts', 'E_demand', 'E_PV', 'Req_kWh', 'Req_kWh_DHW',
       'Set_T', 'Temp', 'Temp_supply', 'Temp_supply_tank', 'T_aux_supply',
       'COP_tank', 'COP_SH', 'COP_DHW', 'E_demand_hp_pv_dhw', 'E_demand_hp_pv',
       'E_demand_pv', 'TSC', 'DSC', 'ISC', 'CU', 'EFC_nolifetime', 'BS', 'LS1',
       'LS2', 'LS3', 'LS4', 'LS5', 'PS1', 'PS2', 'PS3', 'PS4', 'PS5',
       'App_comb', 'conf', 'Capacity', '

In [62]:
df1.head()

,E_PV_batt,E_PV_bu,E_PV_budhw,E_PV_curt,E_PV_grid,E_PV_hp,E_PV_hpdhw,E_PV_load,E_batt_bu,E_batt_budhw,...,SOC_mean,P_drained_max,P_injected_max,last_cap,cap_fading,last_SOH,cycle_to_total,Bill,Bill_noexp,P_max_mean
0,0.0,0.000000,1.564143,414.244502,2547.923453,1305.720924,741.668395,1395.375828,0.0,0.0,...,94.613333,6.375830,3.6,0.980,2.0,0.933516,0.0,1776.810273,1865.733774,NaN
1,0.0,0.000000,1.440126,414.213301,2853.774340,982.351143,744.156980,1410.559796,0.0,0.0,...,94.613333,6.163723,3.6,1.470,2.0,0.933516,0.0,1839.782755,1940.267307,NaN
2,0.0,0.025000,1.440126,414.289395,2989.196051,832.607871,755.457966,1413.483082,0.0,0.0,...,94.613333,9.627475,3.6,1.568,2.0,0.933516,0.0,1942.443522,2049.071710,NaN
3,0.0,0.017466,1.440126,414.358505,3136.744404,689.660632,771.915515,1392.366298,0.0,0.0,...,94.613333,9.617777,3.6,1.568,2.0,0.933516,0.0,2000.444492,2114.162007,NaN
4,0.0,1.054759,1.440126,414.819282,3264.558494,560.896757,782.618321,1381.138245,0.0,0.0,...,94.613333,9.298022,3.6,1.617,2.0,0.933516,0.0,1949.699953,2069.656288,NaN


In [84]:
df1.keys()

Index(['E_PV_batt', 'E_PV_bu', 'E_PV_budhw', 'E_PV_curt', 'E_PV_grid',
       'E_PV_hp', 'E_PV_hpdhw', 'E_PV_load', 'E_batt_bu', 'E_batt_budhw',
       'E_batt_hp', 'E_batt_hpdhw', 'E_batt_load', 'E_bu', 'E_budhw', 'E_char',
       'E_cons', 'E_dis', 'E_grid_batt', 'E_grid_bu', 'E_grid_budhw',
       'E_grid_hp', 'E_grid_hpdhw', 'E_grid_load', 'E_hp', 'E_hpdhw',
       'E_loss_Batt', 'E_loss_conv', 'E_loss_inv', 'E_loss_inv_PV',
       'E_loss_inv_batt', 'E_loss_inv_grid', 'Q_dhwst_hd', 'Q_hp_sh',
       'Q_hp_ts', 'Q_loss_dhwst', 'Q_loss_ts', 'Q_ts', 'Q_ts_delta', 'Q_ts_sh',
       'T_dhwst', 'T_ts', 'E_demand', 'E_PV', 'Req_kWh', 'Req_kWh_DHW',
       'Set_T', 'Temp', 'Temp_supply', 'Temp_supply_tank', 'T_aux_supply',
       'COP_tank', 'COP_SH', 'COP_DHW', 'E_demand_hp_pv_dhw', 'E_demand_hp_pv',
       'E_demand_pv', 'TSC', 'DSC', 'ISC', 'CU', 'EFC_nolifetime', 'BS', 'LS1',
       'LS2', 'LS3', 'LS4', 'LS5', 'PS1', 'PS2', 'PS3', 'PS4', 'PS5',
       'App_comb', 'conf', 'Capacity', '

In [78]:
4536.128449-6006.358563+1453.954572

-16.27554199999986

In [94]:
df1.loc[(df1.house_type!='SFH100')&(df1.Q_loss_ts>0),['Q_loss_ts','Req_kWh']].min()

Q_loss_ts     286.190223
Req_kWh      3814.720686
dtype: float64

In [124]:
df1.loc[(df1.hh==110145456522)&(df1.App_comb==2)&(df1.conf==2)&(df1.house_type=='SFH15'),['house_type','conf','App_comb','Q_ts','Q_ts_sh','Q_hp_sh','Q_hp_ts','Req_kWh','Q_loss_ts','Q_loss_dhwst','Q_ts_delta','E_hp','E_hpdhw','P_drained_max']]

,house_type,conf,App_comb,Q_ts,Q_ts_sh,Q_hp_sh,Q_hp_ts,Req_kWh,Q_loss_ts,Q_loss_dhwst,Q_ts_delta,E_hp,E_hpdhw,P_drained_max
14508,SFH15,2.0,2.0,63047.827674,3814.720686,0.0,4095.359384,3814.720686,287.512361,0.0,-6.873663,1008.841372,0.0,5.26


In [146]:
df2=pd.read_csv('../Output/aggregated_results_jan-Copy1.csv',engine='python',sep=',|;')

In [147]:
df2.shape

(21982, 95)

In [148]:
df2.loc[(df2.conf==1)|(df2.conf==5)].index

Int64Index([    3,     4,    13,    14,    23,    24,    25,    34,    35,
               36,
            ...
            21969, 21970, 21971, 21974, 21975, 21976, 21977, 21978, 21979,
            21980],
           dtype='int64', length=5709)

In [149]:
df2=df2.drop(df2.loc[(df2.conf==1)|(df2.conf==5)].index)

In [150]:
df2.shape

(16273, 95)

In [153]:
df2.to_csv('../Output/aggregated_results.csv',sep=';',index=False)

In [ ]:
df2.drop(df2.loc[df2.conf==])

In [122]:
df2.loc[df2.conf==2,['hh','house_type','conf','App_comb','Q_ts','Q_ts_sh','Q_hp_sh','Q_hp_ts','Req_kWh','Q_loss_ts','Q_loss_dhwst','Q_ts_delta','E_hp','E_hpdhw','P_drained_max']]

,hh,house_type,conf,App_comb,Q_ts,Q_ts_sh,Q_hp_sh,Q_hp_ts,Req_kWh,Q_loss_ts,Q_loss_dhwst,Q_ts_delta,E_hp,E_hpdhw,P_drained_max
29,110698830056,SFH15,2.0,2.0,37489.769933,3814.720686,0.0,3916.365523,3814.720686,100.089666,0.0,1.555171,969.916326,0.0,3.980000
30,110698830059,SFH15,2.0,2.0,36046.800319,3814.720686,0.0,3914.699791,3814.720686,99.640123,0.0,0.338983,967.414925,0.0,6.837706
31,110697929733,SFH15,2.0,2.0,37912.329641,3814.720686,0.0,3913.306073,3814.720686,100.222218,0.0,-1.636831,972.678597,0.0,5.087608
32,110142956061,SFH15,2.0,2.0,38285.178320,3814.720686,0.0,3913.625886,3814.720686,100.337560,0.0,-1.432359,970.432545,0.0,5.080000
33,110144556322,SFH15,2.0,2.0,39865.599517,3814.720686,0.0,3916.221765,3814.720686,100.828180,0.0,0.672899,974.462061,0.0,2.780000
34,110143956137,SFH15,2.0,2.0,37517.997112,3814.720686,0.0,3914.639011,3814.720686,100.096263,0.0,-0.177939,970.224687,0.0,3.980000
35,110145456522,SFH15,2.0,2.0,36202.399103,3814.720686,0.0,3911.474247,3814.720686,99.689733,0.0,-2.936171,966.706744,0.0,5.260000
36,110697929737,SFH15,2.0,2.0,36633.142790,3814.720686,0.0,3913.385436,3814.720686,99.823868,0.0,-1.159117,967.868841,0.0,3.500000
37,110143556103,SFH15,2.0,2.0,37335.357497,3814.720686,0.0,3917.744827,3814.720686,100.041242,0.0,2.982899,969.253263,0.0,4.960000
38,110696328882,SFH15,2.0,2.0,35631.380338,3814.720686,0.0,3915.105829,3814.720686,99.511218,0.0,0.873926,965.404313,0.0,1.820000


In [97]:
df1.loc[:,['house_type','conf','Q_ts','Q_ts_sh','Q_hp_sh','Q_hp_ts','Req_kWh','Q_loss_ts','Q_ts_delta','E_hp','E_hpdhw']]

,house_type,conf,Q_ts,Q_ts_sh,Q_hp_sh,Q_hp_ts,Req_kWh,Q_loss_ts,Q_ts_delta,E_hp,E_hpdhw
0,SFH100,3.0,1314.373199,20553.320576,0.000000,20729.274689,20553.320576,197.193421,-21.239308,6859.670630,1427.509722
1,SFH100,3.0,40951.085498,20553.320576,0.000000,20741.570029,20553.320576,209.488761,-21.239308,7122.955807,1421.469980
2,SFH100,3.0,72382.587062,20553.320576,0.000000,20751.309070,20553.320576,219.227802,-21.239308,7218.390750,1417.262673
3,SFH100,1.0,14504.366166,8109.063959,12444.256617,8209.181841,20553.320576,113.919468,-13.801586,7250.925103,1410.446548
4,SFH100,1.0,0.000000,6006.358563,14546.962013,4536.128449,20553.320576,-1453.954572,-16.275542,6778.220736,1406.961318
5,SFH45,0.0,0.000000,0.000000,9136.942463,0.000000,9136.942463,0.000000,0.000000,2249.572998,0.000000
6,SFH100,0.0,0.000000,0.000000,20553.320576,0.000000,20553.320576,0.000000,0.000000,6700.603645,0.000000
7,SFH15,0.0,0.000000,0.000000,3814.720686,0.000000,3814.720686,0.000000,0.000000,878.449942,0.000000
8,SFH15,0.0,0.000000,0.000000,3814.720686,0.000000,3814.720686,0.000000,0.000000,878.660929,0.000000
9,SFH45,0.0,0.000000,0.000000,9136.942463,0.000000,9136.942463,0.000000,0.000000,2249.822922,0.000000
